In [2]:
import src.paths

In [18]:
DATE = '2022-09-16'
CHAN = 30
THETA_MIN = 2
THETA_MAX = 15
LGAMM_MIN = 30
LGAMM_MAX = 60
HGAMM_MIN = 60
HGAMM_MAX = 100

In [13]:
%%time
csd = pd.read_parquet(src.paths.DATA / DATE / f'{DATE}-hpc-csd-channel-0-59.parquet')
csd = csd.loc[0:7200 - 1 / 1250, CHAN] * 0.001

CPU times: user 3.86 s, sys: 4.08 s, total: 7.94 s
Wall time: 12.4 s


In [20]:
x = csd.values
sos1 = scipy.signal.butter(5, Wn=[THETA_MIN, THETA_MAX], fs=1250, btype='band', output='sos')
sos2 = scipy.signal.butter(5, Wn=[LGAMM_MIN, LGAMM_MAX], fs=1250, btype='band', output='sos')
sos3 = scipy.signal.butter(5, Wn=[HGAMM_MIN, HGAMM_MAX], fs=1250, btype='band', output='sos')
theta = scipy.signal.sosfiltfilt(sos1, x, axis=0)
lgamm = scipy.signal.sosfiltfilt(sos2, x, axis=0)
hgamm = scipy.signal.sosfiltfilt(sos3, x, axis=0)